# Build Graph For The Tutorial

This notebook can work for any root node, the default is `Q2685` for Schwarzenegger

In [1]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd
from IPython.display import display, HTML

import papermill as pm

sys.path.insert(0,'../..')
from kgtk.configure_kgtk_notebooks import ConfigureKGTK

In [2]:
# Parameters
kgtk_path = "/Users/pedroszekely/Documents/GitHub/kgtk"

# Folder on local machine where to create the output and temporary folders
input_path = "/data3/rogers/kgtk/gd/kgtk_public_graphs/cache/wikidata-20210215/data/"
output_path = "/data1/rogers/kgtk/tutorial/"

input_path = "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/"
output_path = "/Users/pedroszekely/Downloads/kypher/projects"

project_name = "build-tutorial"
root = "Q2685"

Put the root q-node in the environment variable `ROOT`

In [3]:
os.environ['ROOT'] = root

In [4]:
files = [
    "claims",
    "item",
    "wikibase_property",
    "datatypes",
    "qualifiers",
    "p31",
    "p279",
    "p279star",
    "quantity",
    "time",
    "external_id",
    "globe_coordinate",
    "monolingualtext",
    "string",
    "label",
    "alias",
    "description"
]
ck = ConfigureKGTK(files, kgtk_path=kgtk_path)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name)

User home: /Users/pedroszekely
Current dir: /Users/pedroszekely/Documents/GitHub/kgtk/tutorial/build-kg
KGTK dir: /Users/pedroszekely/Documents/GitHub/kgtk
Use-cases dir: /Users/pedroszekely/Documents/GitHub/kgtk/use-cases


In [5]:
os.environ['KGTK_LABEL_FILE'] = "{}".format(os.environ['label']) 

In [6]:
ck.print_env_variables()

kgtk: kgtk
GRAPH: /Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data/
STORE: /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/temp.build-tutorial/wikidata.sqlite3.db
USE_CASES_DIR: /Users/pedroszekely/Documents/GitHub/kgtk/use-cases
OUT: /Users/pedroszekely/Downloads/kypher/projects/build-tutorial
EXAMPLES_DIR: /Users/pedroszekely/Documents/GitHub/kgtk/examples
TEMP: /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/temp.build-tutorial
kypher: kgtk query --graph-cache /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/temp.build-tutorial/wikidata.sqlite3.db
claims: /Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data//claims.tsv.gz
item: /Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data//claims.wikibase-item.tsv.gz
wikibase_property: /Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data//claims.wikibase-property.tsv.gz
datatypes: /Volumes/GoogleDrive/Shared drives/KGTK/datas

## Define a custom location for the store when working with full Wikidata so that I can reuse it

In [7]:
os.environ['STORE'] = "/data1/rogers/kgtk/tutorial/wikidata.sqlite3.db"
os.environ['STORE'] = "/Users/pedroszekely/Downloads/kypher/wikidata.sqlite3.db"

Turn on debugging for kypher

In [8]:
os.environ['kypher'] = "kgtk --debug query --graph-cache " + os.environ['STORE']

In [9]:
!echo "$kypher"

kgtk --debug query --graph-cache /Users/pedroszekely/Downloads/kypher/wikidata.sqlite3.db


Load all my files into the kypher cache so that all graph aliases are defined

In [10]:
ck.load_files_into_cache()

kgtk --debug query --graph-cache /Users/pedroszekely/Downloads/kypher/wikidata.sqlite3.db -i "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data//claims.tsv.gz" --as claims  -i "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data//claims.wikibase-item.tsv.gz" --as item  -i "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data//claims.wikibase-property.tsv.gz" --as wikibase_property  -i "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data//metadata.property.datatypes.tsv.gz" --as datatypes  -i "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data//qualifiers.tsv.gz" --as qualifiers  -i "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data//derived.P31.tsv.gz" --as p31  -i "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data//derived.P279.tsv.gz" --as p279  -i "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidata-20210215/data//derived.P279star.ts

In [11]:
%cd {os.environ['OUT']}

/Users/pedroszekely/Downloads/kypher/projects/build-tutorial


# Approach:
- Select a subgraph of full Wikidata that includes people (Q5), organizations (Q43229), geographic regions (Q82794), awards (Q618779) and role (Q4897819) -- otheriwe we get male but do not get female. This graph contains all edges that connect instances of the target classes listed above. Output the graph using a single relation we call `link`.
- Starting from Schwarzenegger Q2685, compute reachable nodes in the graph computed in the previous step. This step will produce the collection of nodes that will be part of the Schwarzenegger graph.
- Extract from Wikidata all the edges that connect nodes from the previous step.
- Extract from Wikidata the time, quantity, monolingual and string properties.
- Extract from Wikidata the qualifiers for the edges computed in the previous steps.
- Extract from Wikidata the labels, aliases and descriptions for the Schwarzenegger nodes.

## Extract a subset of Wikidata to use as the base for the Schewarzenegger graph

This query takes a really long time, so don't re-execute unless you have to.

In [12]:
%%time
!$kypher -i p31 -i item -i p279star \
--match ' \
    p31: (n1)-[]->(n1_class), \
    item: (n1)-[l]->(n2), \
    p31: (n2)-[]->(n2_class), \
    p279star: (n1_class)-[]->(n1_superclass), \
    p279star: (n2_class)-[]->(n2_superclass)' \
--where 'n1_superclass in ["Q11424", "Q5", "Q43229", "Q82794", "Q618779"] and n2_superclass in ["Q11424", "Q5", "Q43229", "Q82794", "Q618779", "Q4897819"]' \
--return 'distinct n1 as node1, "link" as label, n2 as node2, l as id' \
-o "$TEMP"/item.per.org.cw.geo.award.link.tsv.gz 

[2021-10-09 20:17:56 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_6_c1."node1" "_aLias.node1", ? "_aLias.label", graph_2_c2."node2" "_aLias.node2", graph_2_c2."id" "_aLias.id"
     FROM graph_2 AS graph_2_c2
     INNER JOIN graph_6 AS graph_6_c1, graph_6 AS graph_6_c3, graph_8 AS graph_8_c4, graph_8 AS graph_8_c5
     ON graph_2_c2."node2" = graph_6_c3."node1"
        AND graph_6_c1."node1" = graph_2_c2."node1"
        AND graph_6_c1."node2" = graph_8_c4."node1"
        AND graph_6_c3."node2" = graph_8_c5."node1"
        AND ((graph_8_c4."node2" IN (?, ?, ?, ?, ?)) AND (graph_8_c5."node2" IN (?, ?, ?, ?, ?, ?)))
  PARAS: ['link', 'Q11424', 'Q5', 'Q43229', 'Q82794', 'Q618779', 'Q11424', 'Q5', 'Q43229', 'Q82794', 'Q618779', 'Q4897819']
---------------------------------------------
CPU times: user 2min 42s, sys: 47.7 s, total: 3min 30s
Wall time: 3h 51min 46s


In the original graph there are qualifier values that we want to follow in the reachablity search. To do so, we will create `link` edges between the qualifier and the value of the statement on which the qualifier is defined.

In [13]:
%%time
!$kypher -i qualifiers -i datatypes -i "$TEMP"/item.per.org.cw.geo.award.link.tsv.gz --as links \
--match ' \
    links: ()-[l]->(n2), \
    qualifiers: (l)-[q {label: property}]->(qualifier), \
    datatypes: (property)-[:datatype]->(datatype) \
    ' \
--where 'datatype in ["wikibase-item"]' \
--return 'n2 as node1, "link" as label, qualifier as node2' \
/ add-id --id-style wikidata \
/ cat -i - -i "$TEMP"/item.per.org.cw.geo.award.link.tsv.gz \
-o "$TEMP"/item.per.org.cw.geo.award.link.qualifier.tsv.gz

[2021-10-10 00:09:44 sqlstore]: DROP graph data table graph_18 from links
[2021-10-10 00:10:56 sqlstore]: IMPORT graph directly into table graph_28 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/temp.build-tutorial/item.per.org.cw.geo.award.link.tsv.gz ...
[2021-10-10 00:15:10 query]: SQL Translation:
---------------------------------------------
  SELECT graph_28_c1."node2" "_aLias.node1", ? "_aLias.label", graph_5_c2."node2" "_aLias.node2"
     FROM graph_28 AS graph_28_c1
     INNER JOIN graph_4 AS graph_4_c3, graph_5 AS graph_5_c2
     ON graph_28_c1."id" = graph_5_c2."node1"
        AND graph_4_c3."node1" = graph_5_c2."label"
        AND graph_4_c3."label" = ?
        AND graph_5_c2."label" = graph_4_c3."node1"
        AND (graph_4_c3."node2" IN (?))
  PARAS: ['link', 'datatype', 'wikibase-item']
---------------------------------------------
[2021-10-10 00:15:10 sqlstore]: CREATE INDEX on table graph_28 column id ...
[2021-10-10 00:15:53 sqlstore]: ANALYZE INDEX

Starting from `ROOT` traverse links forward in breadfirst mode up to a fixed number of levels to build the graph

In [14]:
%%time
!$kgtk reachable-nodes \
    --root $ROOT \
    --prop link \
    --label "reachable" \
    --selflink \
    --breadth-first --depth-limit 3 \
    -i "$TEMP"/item.per.org.cw.geo.award.link.qualifier.tsv.gz  \
    -o "$TEMP"/root.reachable.per.org.cw.geo.award.tsv.gz

CPU times: user 3.84 s, sys: 1.18 s, total: 5.02 s
Wall time: 6min 41s


In [15]:
!$kgtk head -i "$TEMP"/root.reachable.per.org.cw.geo.award.tsv.gz

node1	label	node2
Q2685	reachable	Q2685
Q2685	reachable	Q12158205
Q2685	reachable	Q170564
Q2685	reachable	Q1765879
Q2685	reachable	Q28754213
Q2685	reachable	Q1976616
Q2685	reachable	Q551327
Q2685	reachable	Q12325509
Q2685	reachable	Q30331794
Q2685	reachable	Q557584


Index the resulting file in kypher

In [16]:
!$kypher -i $TEMP/root.reachable.per.org.cw.geo.award.tsv.gz --as root_nodes --limit 2

[2021-10-10 00:54:29 sqlstore]: DROP graph data table graph_19 from root_nodes
[2021-10-10 00:54:29 sqlstore]: IMPORT graph directly into table graph_29 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/temp.build-tutorial/root.reachable.per.org.cw.geo.award.tsv.gz ...
[2021-10-10 00:54:29 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_29 AS graph_29_c1
     LIMIT ?
  PARAS: [2]
---------------------------------------------
node1	label	node2
Q2685	reachable	Q2685
Q2685	reachable	Q12158205


## Build initial graph containing the item edges

Figure out which properties are used so so that we can add them as node1s and get all the info about them.

In [17]:
%%time
!$kypher -i root_nodes -i datatypes -i claims \
--match ' \
    root_nodes: ()-[]->(n1), \
    claims: (n1)-[l {label: property}]->(), \
    datatypes: (property)-[:datatype]->(datatype) \
    ' \
--where 'datatype in ["wikibase-item", "string", "quantity", "time", "monolingualtext"]' \
--return 'distinct "root" as node1, "link" as label, property as node2' \
-o "$TEMP"/root.nodes.property.tsv.gz

[2021-10-10 00:54:29 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT ? "_aLias.node1", ? "_aLias.label", graph_4_c3."node1" "_aLias.node2"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_29 AS graph_29_c1, graph_4 AS graph_4_c3
     ON graph_29_c1."node2" = graph_1_c2."node1"
        AND graph_4_c3."node1" = graph_1_c2."label"
        AND graph_1_c2."label" = graph_4_c3."node1"
        AND graph_4_c3."label" = ?
        AND (graph_4_c3."node2" IN (?, ?, ?, ?, ?))
  PARAS: ['root', 'link', 'datatype', 'wikibase-item', 'string', 'quantity', 'time', 'monolingualtext']
---------------------------------------------
[2021-10-10 00:54:29 sqlstore]: CREATE INDEX on table graph_29 column node2 ...
[2021-10-10 00:54:29 sqlstore]: ANALYZE INDEX on table graph_29 column node2 ...
CPU times: user 295 ms, sys: 95.3 ms, total: 390 ms
Wall time: 30.8 s


Concatenate the new nodes with the ones we found via reachability

In [18]:
!kgtk cat -i "$TEMP"/root.nodes.property.tsv.gz -i "$TEMP"/root.reachable.per.org.cw.geo.award.tsv.gz \
-o "$TEMP"/root.nodes.all.tsv.gz

Print number of nodes that we have so far for the new graph

In [19]:
!zcat < "$TEMP"/root.nodes.all.tsv.gz | wc -l

   42360


Update the Kypher database

In [20]:
!$kypher -i "$TEMP"/root.nodes.all.tsv.gz --as root_nodes --limit 2

[2021-10-10 00:55:01 sqlstore]: DROP graph data table graph_29 from root_nodes
[2021-10-10 00:55:01 sqlstore]: IMPORT graph directly into table graph_29 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/temp.build-tutorial/root.nodes.all.tsv.gz ...
[2021-10-10 00:55:01 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_29 AS graph_29_c1
     LIMIT ?
  PARAS: [2]
---------------------------------------------
node1	label	node2
root	link	P1082
root	link	P112


Extract the item to item edges connecting the nodes in the new graph

In [21]:
%%time
!$kypher -i root_nodes -i item \
--match ' \
    root_nodes: ()-[]->(n1), \
    root_nodes: ()-[]->(n2), \
    item: (n1)-[l]->(n2) \
    ' \
--return 'distinct n1 as node1, l.label as label, n2 as node2, l as id' \
/ sort \
-o $OUT/root.graph.item.tsv.gz

[2021-10-10 00:55:02 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_29_c1."node2" "_aLias.node1", graph_2_c3."label" "_aLias.label", graph_29_c2."node2" "_aLias.node2", graph_2_c3."id" "_aLias.id"
     FROM graph_2 AS graph_2_c3
     INNER JOIN graph_29 AS graph_29_c1, graph_29 AS graph_29_c2
     ON graph_29_c1."node2" = graph_2_c3."node1"
        AND graph_29_c2."node2" = graph_2_c3."node2"
  PARAS: []
---------------------------------------------
[2021-10-10 00:55:02 sqlstore]: CREATE INDEX on table graph_29 column node2 ...
[2021-10-10 00:55:02 sqlstore]: ANALYZE INDEX on table graph_29 column node2 ...
CPU times: user 265 ms, sys: 85.9 ms, total: 351 ms
Wall time: 26.7 s


Add to the kypher database

In [22]:
!$kypher -i $OUT/root.graph.item.tsv.gz --as rootitems --limit 2

[2021-10-10 00:55:28 sqlstore]: DROP graph data table graph_23 from rootitems
[2021-10-10 00:55:28 sqlstore]: IMPORT graph directly into table graph_30 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/root.graph.item.tsv.gz ...
[2021-10-10 00:55:30 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_30 AS graph_30_c1
     LIMIT ?
  PARAS: [2]
---------------------------------------------
node1	label	node2	id
P1001	P1855	Q11696	P1001-P1855-Q11696-cdbf391b-0
P1001	P1855	Q12371988	P1001-P1855-Q12371988-12c10bc0-0


## Extract the other types of edges

Extract the quantities

In [23]:
%%time
!$kypher -i quantity -i root_nodes \
--match ' \
    root_nodes: ()-[]->(n1), \
    quantity: (n1)-[l]->(n2) \
    ' \
--return 'distinct n1 as node1, l.label as label, n2 as node2, l as id' \
/ sort \
-o $OUT/root.graph.quantity.tsv.gz

[2021-10-10 00:55:30 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_29_c1."node2" "_aLias.node1", graph_9_c2."label" "_aLias.label", graph_9_c2."node2" "_aLias.node2", graph_9_c2."id" "_aLias.id"
     FROM graph_29 AS graph_29_c1
     INNER JOIN graph_9 AS graph_9_c2
     ON graph_29_c1."node2" = graph_9_c2."node1"
  PARAS: []
---------------------------------------------
CPU times: user 68.4 ms, sys: 25 ms, total: 93.4 ms
Wall time: 6.42 s


Extract the time edges

In [24]:
%%time
!$kypher -i time -i root_nodes \
--match ' \
    root_nodes: ()-[]->(n1), \
    time: (n1)-[l]->(n2) \
    ' \
--return 'distinct n1 as node1, l.label as label, n2 as node2, l as id' \
/ sort \
-o $OUT/root.graph.time.tsv.gz

[2021-10-10 00:55:37 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_29_c1."node2" "_aLias.node1", graph_10_c2."label" "_aLias.label", graph_10_c2."node2" "_aLias.node2", graph_10_c2."id" "_aLias.id"
     FROM graph_10 AS graph_10_c2
     INNER JOIN graph_29 AS graph_29_c1
     ON graph_29_c1."node2" = graph_10_c2."node1"
  PARAS: []
---------------------------------------------
CPU times: user 90.8 ms, sys: 34.1 ms, total: 125 ms
Wall time: 9.25 s


Extract the monolingual text edges

In [25]:
%%time
!$kypher -i monolingualtext -i root_nodes \
--match ' \
    root_nodes: ()-[]->(n1), \
    monolingualtext: (n1)-[l]->(n2) \
    ' \
--return 'distinct n1 as node1, l.label as label, n2 as node2, l as id' \
/ sort \
-o $OUT/root.graph.monolingual.tsv.gz

[2021-10-10 00:55:46 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_29_c1."node2" "_aLias.node1", graph_13_c2."label" "_aLias.label", graph_13_c2."node2" "_aLias.node2", graph_13_c2."id" "_aLias.id"
     FROM graph_13 AS graph_13_c2
     INNER JOIN graph_29 AS graph_29_c1
     ON graph_29_c1."node2" = graph_13_c2."node1"
  PARAS: []
---------------------------------------------
CPU times: user 78.1 ms, sys: 29.3 ms, total: 107 ms
Wall time: 7.56 s


Extract the string edges

In [26]:
%%time
!$kypher -i string -i root_nodes \
--match ' \
    root_nodes: ()-[]->(n1), \
    string: (n1)-[l]->(n2) \
    ' \
--return 'distinct n1 as node1, l.label as label, n2 as node2, l as id' \
/ sort \
-o $OUT/root.graph.string.tsv.gz

[2021-10-10 00:55:54 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_29_c1."node2" "_aLias.node1", graph_14_c2."label" "_aLias.label", graph_14_c2."node2" "_aLias.node2", graph_14_c2."id" "_aLias.id"
     FROM graph_14 AS graph_14_c2
     INNER JOIN graph_29 AS graph_29_c1
     ON graph_29_c1."node2" = graph_14_c2."node1"
  PARAS: []
---------------------------------------------
CPU times: user 133 ms, sys: 46.4 ms, total: 180 ms
Wall time: 12.7 s


Extract external identifiers NEW

In [27]:
%%time
!$kypher -i external_id -i root_nodes \
--match ' \
    root_nodes: ()-[]->(n1), \
    external_id: (n1)-[l]->(n2) \
    ' \
--return 'distinct n1 as node1, l.label as label, n2 as node2, l as id' \
/ sort \
-o $OUT/root.graph.external_ids.tsv.gz

[2021-10-10 00:56:07 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_29_c1."node2" "_aLias.node1", graph_11_c2."label" "_aLias.label", graph_11_c2."node2" "_aLias.node2", graph_11_c2."id" "_aLias.id"
     FROM graph_11 AS graph_11_c2
     INNER JOIN graph_29 AS graph_29_c1
     ON graph_29_c1."node2" = graph_11_c2."node1"
  PARAS: []
---------------------------------------------
CPU times: user 236 ms, sys: 78.1 ms, total: 314 ms
Wall time: 23 s


## Complete the graph

Add external_ids

In [28]:
%%time
!kgtk cat \
-i $OUT/root.graph.item.tsv.gz \
-i $OUT/root.graph.quantity.tsv.gz \
-i $OUT/root.graph.time.tsv.gz \
-i $OUT/root.graph.monolingual.tsv.gz \
-i $OUT/root.graph.string.tsv.gz \
-i $OUT/root.graph.external_ids.tsv.gz \
-o $OUT/root.graph.item.quantity.time.monolingual.string.tsv.gz 

!$kypher -i $OUT/root.graph.item.quantity.time.monolingual.string.tsv.gz --as rootbase --limit 2

[2021-10-10 00:56:37 sqlstore]: DROP graph data table graph_24 from rootbase
[2021-10-10 00:56:39 sqlstore]: IMPORT graph directly into table graph_24 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/root.graph.item.quantity.time.monolingual.string.tsv.gz ...
[2021-10-10 00:56:43 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_24 AS graph_24_c1
     LIMIT ?
  PARAS: [2]
---------------------------------------------
node1	label	node2	id
P1001	P1855	Q11696	P1001-P1855-Q11696-cdbf391b-0
P1001	P1855	Q12371988	P1001-P1855-Q12371988-12c10bc0-0
CPU times: user 157 ms, sys: 57.4 ms, total: 214 ms
Wall time: 14.7 s


### Collect all the properties

Get edges for the properties

In [29]:
%%time
!$kypher -i rootbase -i wikibase_property \
--match ' \
    rootbase: ()-[l {label: property}]->(), \
    wikibase_property: (property)-[lp]->(n) \
    ' \
--return 'distinct property as node1, lp.label as label, n as node2, lp as id' \
/ sort \
-o $OUT/root.graph.property.tsv.gz

[2021-10-10 00:56:44 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_3_c2."node1" "_aLias.node1", graph_3_c2."label" "_aLias.label", graph_3_c2."node2" "_aLias.node2", graph_3_c2."id" "_aLias.id"
     FROM graph_24 AS graph_24_c1
     INNER JOIN graph_3 AS graph_3_c2
     ON graph_3_c2."node1" = graph_24_c1."label"
        AND graph_24_c1."label" = graph_3_c2."node1"
  PARAS: []
---------------------------------------------
[2021-10-10 00:56:44 sqlstore]: CREATE INDEX on table graph_24 column label ...
[2021-10-10 00:56:45 sqlstore]: ANALYZE INDEX on table graph_24 column label ...
CPU times: user 39.1 ms, sys: 17.7 ms, total: 56.8 ms
Wall time: 3.47 s


Update the base

In [30]:
%%time
!kgtk cat \
-i $OUT/root.graph.item.quantity.time.monolingual.string.tsv.gz \
-i $OUT/root.graph.property.tsv.gz \
/ compact \
-o $OUT/root.graph.item.quantity.time.monolingual.string.property.tsv.gz 

!$kypher -i $OUT/root.graph.item.quantity.time.monolingual.string.property.tsv.gz --as rootbase --limit 2

[2021-10-10 00:57:08 sqlstore]: DROP graph data table graph_24 from rootbase
[2021-10-10 00:57:09 sqlstore]: IMPORT graph directly into table graph_24 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/root.graph.item.quantity.time.monolingual.string.property.tsv.gz ...
[2021-10-10 00:57:14 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_24 AS graph_24_c1
     LIMIT ?
  PARAS: [2]
---------------------------------------------
node1	label	node2	id
P1001	P1647	P276	P1001-P1647-P276-e4e44f83-0
P1001	P1659	P1269	P1001-P1659-P1269-785921cd-0
CPU times: user 279 ms, sys: 93.9 ms, total: 373 ms
Wall time: 27 s


### Compute qualifiers

In [31]:
%%time
!$kypher -i qualifiers -i rootbase \
--match ' \
    rootbase: ()-[l]->(), \
    qualifiers: (l)-[lq {label: property}]->(n) \
    ' \
--return 'distinct l as node1, property as label, n as node2, lq as id' \
/ sort \
-o $OUT/root.graph.qualifiers.tsv.gz

[2021-10-10 00:57:17 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_24_c1."id" "_aLias.node1", graph_5_c2."label" "_aLias.label", graph_5_c2."node2" "_aLias.node2", graph_5_c2."id" "_aLias.id"
     FROM graph_24 AS graph_24_c1
     INNER JOIN graph_5 AS graph_5_c2
     ON graph_24_c1."id" = graph_5_c2."node1"
        AND graph_5_c2."label" = graph_5_c2."label"
  PARAS: []
---------------------------------------------
[2021-10-10 00:57:17 sqlstore]: CREATE INDEX on table graph_24 column id ...
[2021-10-10 00:57:17 sqlstore]: ANALYZE INDEX on table graph_24 column id ...
CPU times: user 278 ms, sys: 92.7 ms, total: 370 ms
Wall time: 25.3 s


Update the base again

In [32]:
%%time
!kgtk cat \
-i $OUT/root.graph.item.quantity.time.monolingual.string.property.tsv.gz \
-i $OUT/root.graph.qualifiers.tsv.gz \
/ compact \
-o $OUT/root.graph.item.quantity.time.monolingual.string.property.qualifiers.tsv.gz 

!$kypher -i $OUT/root.graph.item.quantity.time.monolingual.string.property.qualifiers.tsv.gz --as rootbase --limit 2

[2021-10-10 00:58:07 sqlstore]: DROP graph data table graph_24 from rootbase
[2021-10-10 00:58:08 sqlstore]: IMPORT graph directly into table graph_24 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/root.graph.item.quantity.time.monolingual.string.property.qualifiers.tsv.gz ...
[2021-10-10 00:58:15 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_24 AS graph_24_c1
     LIMIT ?
  PARAS: [2]
---------------------------------------------
node1	label	node2	id
P1001	P1647	P276	P1001-P1647-P276-e4e44f83-0
P1001	P1659	P1269	P1001-P1659-P1269-785921cd-0
CPU times: user 396 ms, sys: 130 ms, total: 526 ms
Wall time: 36 s


### Add the units

Find all values of quantity properties, and get the units defined for them.

> `kgtk_quantity_wd_units` throws an exception when it gets a quantity without units, so we have to hack around that using grep.

In [33]:
%%time
!$kypher -i datatypes -i rootbase \
--match ' \
    rootbase: ()-[l {label: property}]->(n2), \
    datatypes: (property)-[:datatype]->(datatype) \
    ' \
--where 'datatype in ["quantity"]' \
--return 'distinct n2' \
| grep Q > "$TEMP"/units.noheader.tsv

!echo -e "node1" | cat - "$TEMP"/units.noheader.tsv > "$TEMP"/quantities.units.tsv

[2021-10-10 00:58:16 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_24_c1."node2"
     FROM graph_24 AS graph_24_c1
     INNER JOIN graph_4 AS graph_4_c2
     ON graph_4_c2."node1" = graph_24_c1."label"
        AND graph_24_c1."label" = graph_4_c2."node1"
        AND graph_4_c2."label" = ?
        AND (graph_4_c2."node2" IN (?))
  PARAS: ['datatype', 'quantity']
---------------------------------------------
[2021-10-10 00:58:16 sqlstore]: CREATE INDEX on table graph_24 column label ...
[2021-10-10 00:58:17 sqlstore]: ANALYZE INDEX on table graph_24 column label ...
CPU times: user 24.2 ms, sys: 16.8 ms, total: 41 ms
Wall time: 1.96 s


In [34]:
%%time
!$kypher -i "$TEMP"/quantities.units.tsv \
--match '(quantity)' \
--return 'distinct kgtk_quantity_wd_units(quantity) as node1' \
-o "$TEMP"/units.tsv

[2021-10-10 00:58:18 sqlstore]: DROP graph data table graph_25 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/temp.build-tutorial/quantities.units.tsv
[2021-10-10 00:58:18 sqlstore]: IMPORT graph directly into table graph_25 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/temp.build-tutorial/quantities.units.tsv ...
[2021-10-10 00:58:18 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT kgtk_quantity_wd_units(graph_25_c1."node1") "_aLias.node1"
     FROM graph_25 AS graph_25_c1
  PARAS: []
---------------------------------------------
CPU times: user 8.65 ms, sys: 8.12 ms, total: 16.8 ms
Wall time: 674 ms


Now that we have the units in a file, we can get all the properties we want about them

In [35]:
%%time
!$kypher -i "$TEMP"/units.tsv -i item -i datatypes \
--match ' \
    units: (unit), \
    datatypes: (property)-[:datatype]->(datatype), \
    item: (unit)-[l {label: property}]->(n2) \
    ' \
--where 'datatype in ["wikibase-item", "string", "quantity", "time", "monolingualtext"]' \
--return 'distinct unit as node1, property as label, n2 as node2, l as id' \
/ cat -i - -i $OUT/root.graph.item.quantity.time.monolingual.string.property.qualifiers.tsv.gz\
/ compact \
-o $OUT/root.graph.item.quantity.time.monolingual.string.property.qualifiers.units.tsv.gz \

!$kypher -i $OUT/root.graph.item.quantity.time.monolingual.string.property.qualifiers.units.tsv.gz --as rootbase --limit 2

[2021-10-10 00:58:19 sqlstore]: DROP graph data table graph_26 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/temp.build-tutorial/units.tsv
[2021-10-10 00:58:19 sqlstore]: IMPORT graph directly into table graph_26 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/temp.build-tutorial/units.tsv ...
[2021-10-10 00:58:19 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_26_c1."node1" "_aLias.node1", graph_4_c2."node1" "_aLias.label", graph_2_c3."node2" "_aLias.node2", graph_2_c3."id" "_aLias.id"
     FROM graph_2 AS graph_2_c3
     INNER JOIN graph_26 AS graph_26_c1, graph_4 AS graph_4_c2
     ON graph_26_c1."node1" = graph_2_c3."node1"
        AND graph_4_c2."node1" = graph_2_c3."label"
        AND graph_2_c3."label" = graph_4_c2."node1"
        AND graph_4_c2."label" = ?
        AND (graph_4_c2."node2" IN (?, ?, ?, ?, ?))
  PARAS: ['datatype', 'wikibase-item', 'string', 'quantity', 'time', 'monolingualtext']


### Make sure that every q-node has at least P31 and P279
need to do it twice, once for node1 and once for node2

In [36]:
%%time
!$kypher -i rootbase -i claims \
--match 'rootbase: (n)-[]->(), claims: (n)-[l {label: property}]->(n2)' \
--where 'property in ["P31", "P279"]' \
--return 'distinct n as node1, property as label, n2 as node2, l as id' \
-o "$TEMP"/root.node1.P31.P279.tsv.gz

[2021-10-10 00:58:54 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_24_c1."node1" "_aLias.node1", graph_1_c2."label" "_aLias.label", graph_1_c2."node2" "_aLias.node2", graph_1_c2."id" "_aLias.id"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_24 AS graph_24_c1
     ON graph_24_c1."node1" = graph_1_c2."node1"
        AND graph_1_c2."label" = graph_1_c2."label"
        AND (graph_1_c2."label" IN (?, ?))
  PARAS: ['P31', 'P279']
---------------------------------------------
[2021-10-10 00:58:54 sqlstore]: CREATE INDEX on table graph_24 column node1 ...
[2021-10-10 00:58:55 sqlstore]: ANALYZE INDEX on table graph_24 column node1 ...
CPU times: user 11.8 s, sys: 3.52 s, total: 15.4 s
Wall time: 17min 41s


In [37]:
%%time
!$kypher -i rootbase -i claims \
--match 'rootbase: ()-[]->(n), claims: (n)-[l {label: property}]->(n2)' \
--where 'property in ["P31", "P279"]' \
--return 'distinct n as node1, property as label, n2 as node2, l as id' \
-o "$TEMP"/root.node2.P31.P279.tsv.gz

[2021-10-10 01:16:36 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_24_c1."node2" "_aLias.node1", graph_1_c2."label" "_aLias.label", graph_1_c2."node2" "_aLias.node2", graph_1_c2."id" "_aLias.id"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_24 AS graph_24_c1
     ON graph_24_c1."node2" = graph_1_c2."node1"
        AND graph_1_c2."label" = graph_1_c2."label"
        AND (graph_1_c2."label" IN (?, ?))
  PARAS: ['P31', 'P279']
---------------------------------------------
[2021-10-10 01:16:36 sqlstore]: CREATE INDEX on table graph_24 column node2 ...
[2021-10-10 01:16:38 sqlstore]: ANALYZE INDEX on table graph_24 column node2 ...
CPU times: user 12.3 s, sys: 3.73 s, total: 16 s
Wall time: 18min 1s


Recreate the base file NEW
> the output file should have the `.units` segment in the name, but I didnt' add it so that I don't have to modify all the other ocmmands
> a better design for the file names would not have this problem

In [38]:
%%time
!kgtk cat \
-i $OUT/root.graph.item.quantity.time.monolingual.string.property.qualifiers.units.tsv.gz \
-i "$TEMP"/root.node2.P31.P279.tsv.gz \
/ compact \
-o $OUT/root.graph.item.quantity.time.monolingual.string.property.qualifiers.P31.P279.tsv.gz 

!$kypher -i $OUT/root.graph.item.quantity.time.monolingual.string.property.qualifiers.P31.P279.tsv.gz --as rootbase --limit 2

[2021-10-10 01:35:08 sqlstore]: DROP graph data table graph_24 from rootbase
[2021-10-10 01:35:10 sqlstore]: IMPORT graph directly into table graph_24 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/root.graph.item.quantity.time.monolingual.string.property.qualifiers.P31.P279.tsv.gz ...
[2021-10-10 01:35:17 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_24 AS graph_24_c1
     LIMIT ?
  PARAS: [2]
---------------------------------------------
node1	label	node2	id
P10	P31	Q18610173	P10-P31-Q18610173-85ef4d24-0
P1000	P31	Q18608871	P1000-P31-Q18608871-093affb5-0
CPU times: user 438 ms, sys: 146 ms, total: 584 ms
Wall time: 39.9 s


### Incorporate all nodes up to the top of the class hierarchy
When we do a breath first traversal, we may not follow enough links on the P279 hierarchy to reach the top. We need to do a full traversal on the P279 hierarchy to incorporate all the relevant classes.

Approach:
- Create a graph including P31 and P279 to do the traversal
- Create a file of all the nodes in the Schwarzenneger file to use as roots

In [39]:
%%time
!$kypher -i claims \
--match '(n1)-[l {label:property}]->(n2)' \
--where 'property in ["P31", "P279"]' \
--return 'distinct n1 as node1, "link" as label, n2 as node2' \
-o "$TEMP"/P31.P279.subgraph.tsv.gz

[2021-10-10 01:35:19 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c1."node1" "_aLias.node1", ? "_aLias.label", graph_1_c1."node2" "_aLias.node2"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label" = graph_1_c1."label"
        AND (graph_1_c1."label" IN (?, ?))
  PARAS: ['link', 'P31', 'P279']
---------------------------------------------
CPU times: user 19.3 s, sys: 5.91 s, total: 25.2 s
Wall time: 28min 37s


#### Create the roots

Find roots in node1

> This step is including qualifier ids in node1, which makes reachable nodes have more roots than necessary. Would be nice to eliminate qualifiers here.

In [40]:
%%time
!$kypher -i rootbase \
--match '(n)-[]->()' \
--return 'distinct n as node1' \
-o "$TEMP"/root.node1.tsv.gz

[2021-10-10 02:03:55 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_24_c1."node1" "_aLias.node1"
     FROM graph_24 AS graph_24_c1
  PARAS: []
---------------------------------------------
CPU times: user 81.9 ms, sys: 34 ms, total: 116 ms
Wall time: 6.93 s


Find roots in node2

In [41]:
%%time
!$kypher -i rootbase -i datatypes \
--match ' \
    rootbase: ()-[l {label: property}]->(n), \
    datatypes: (property)-[:datatype]->(datatype) \
    ' \
--where 'datatype in ["wikibase-item"]' \
--return 'distinct n as node1' \
-o "$TEMP"/root.node2.tsv.gz

[2021-10-10 02:04:02 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_24_c1."node2" "_aLias.node1"
     FROM graph_24 AS graph_24_c1
     INNER JOIN graph_4 AS graph_4_c2
     ON graph_4_c2."node1" = graph_24_c1."label"
        AND graph_24_c1."label" = graph_4_c2."node1"
        AND graph_4_c2."label" = ?
        AND (graph_4_c2."node2" IN (?))
  PARAS: ['datatype', 'wikibase-item']
---------------------------------------------
[2021-10-10 02:04:02 sqlstore]: CREATE INDEX on table graph_24 column label ...
[2021-10-10 02:04:03 sqlstore]: ANALYZE INDEX on table graph_24 column label ...
CPU times: user 27.9 ms, sys: 15.7 ms, total: 43.6 ms
Wall time: 2.29 s


Combine the two files to create all the roots

In [42]:
%%time
!$kgtk cat --mode=NONE -i "$TEMP"/root.node1.tsv.gz -i "$TEMP"/root.node2.tsv.gz \
/ compact --mode=NONE --columns node1 \
-o "$TEMP"/root.nodes.tsv.gz

!$kypher -i "$TEMP"/root.nodes.tsv.gz --as rootnode1 --limit 2

[2021-10-10 02:04:08 sqlstore]: DROP graph data table graph_27 from rootnode1
[2021-10-10 02:04:08 sqlstore]: IMPORT graph directly into table graph_27 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/temp.build-tutorial/root.nodes.tsv.gz ...
[2021-10-10 02:04:09 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_27 AS graph_27_c1
     LIMIT ?
  PARAS: [2]
---------------------------------------------
node1
P10
P1000
CPU times: user 58.9 ms, sys: 28.3 ms, total: 87.2 ms
Wall time: 5.06 s


Circumvent a problem in `reachable-nodes` where it does not accept a root file with column header `node1`

In [43]:
%%time
!$kgtk rename-columns -i "$TEMP"/root.nodes.tsv.gz --output-columns id --mode=NONE \
/ compact -o "$TEMP"/root.roots.tsv.gz

CPU times: user 47.7 ms, sys: 19 ms, total: 66.7 ms
Wall time: 3.78 s


Do a depth-first traversal of the P31/P279 graph using as roots all items in the Schewarzenegger graph

In [44]:
%%time
!$kgtk reachable-nodes \
    --rootfile "$TEMP"/root.roots.tsv.gz \
    --rootfilecolumn id \
    --prop link \
    --label "reachable" \
    --selflink \
    -i "$TEMP"/P31.P279.subgraph.tsv.gz \
    -o "$TEMP"/P31.P279.reachable.tsv.gz

CPU times: user 19.9 s, sys: 6.11 s, total: 26 s
Wall time: 30min 11s


Deduplicate the reachable nodes file

In [45]:
%%time
!$kgtk remove-columns -i "$TEMP"/P31.P279.reachable.tsv.gz --columns node1 label \
/ rename-columns --mode=NONE --output-columns node1 \
/ compact --mode=NONE --columns node1 \
-o "$TEMP"/P31.P279.reachable.dedup.tsv.gz

CPU times: user 642 ms, sys: 203 ms, total: 845 ms
Wall time: 1min


Put all the reachable nodes in `rootnode1`

In [46]:
%%time
!$kgtk cat --mode=NONE \
-i "$TEMP"/root.nodes.tsv.gz \
-i "$TEMP"/P31.P279.reachable.dedup.tsv.gz \
/ compact --deduplicate --mode=NONE --columns node1 \
-o "$TEMP"/root.nodes.ontology.tsv.gz

!$kypher -i "$TEMP"/root.nodes.ontology.tsv.gz --as rootnode1 --limit 2

[2021-10-10 02:35:29 sqlstore]: DROP graph data table graph_27 from rootnode1
[2021-10-10 02:35:29 sqlstore]: IMPORT graph directly into table graph_27 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/temp.build-tutorial/root.nodes.ontology.tsv.gz ...
[2021-10-10 02:35:30 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_27 AS graph_27_c1
     LIMIT ?
  PARAS: [2]
---------------------------------------------
node1
P10
P1000
CPU times: user 53.7 ms, sys: 27.2 ms, total: 81 ms
Wall time: 5.17 s


Extract all P31/P279 edges from Wikidata for all the nodes in the new graph and consolidate.

In [47]:
%%time
!$kypher -i claims -i rootnode1 \
--match ' \
    rootnode1: (n1), \
    claims: (n1)-[l {label:property}]->(n2) \
    ' \
--where 'property in ["P31", "P279"]' \
--return 'distinct n1 as node1, property as label, n2 as node2, l as id' \
/ cat -i - -i $OUT/root.graph.item.quantity.time.monolingual.string.property.qualifiers.P31.P279.tsv.gz \
/ compact \
-o $OUT/root.graph.item.quantity.time.monolingual.string.property.qualifiers.P31.P279.ontology.tsv.gz \

!$kypher -i $OUT/root.graph.item.quantity.time.monolingual.string.property.qualifiers.P31.P279.ontology.tsv.gz --as rootbase --limit 2

[2021-10-10 02:35:30 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_27_c1."node1" "_aLias.node1", graph_1_c2."label" "_aLias.label", graph_1_c2."node2" "_aLias.node2", graph_1_c2."id" "_aLias.id"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_27 AS graph_27_c1
     ON graph_27_c1."node1" = graph_1_c2."node1"
        AND graph_1_c2."label" = graph_1_c2."label"
        AND (graph_1_c2."label" IN (?, ?))
  PARAS: ['P31', 'P279']
---------------------------------------------
[2021-10-10 02:35:30 sqlstore]: CREATE INDEX on table graph_27 column node1 ...
[2021-10-10 02:35:31 sqlstore]: ANALYZE INDEX on table graph_27 column node1 ...
[2021-10-10 02:52:25 sqlstore]: DROP graph data table graph_24 from rootbase
[2021-10-10 02:52:28 sqlstore]: IMPORT graph directly into table graph_24 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/root.graph.item.quantity.time.monolingual.string.property.qualifiers.P31.P279.ontology.tsv.

I am not certain about the need for this cell, whether new nodes can appear after adding P31 and P279.

In [48]:
%%time
!$kypher -i rootbase -i datatypes \
--match ' \
    rootbase: ()-[l {label: property}]->(n), \
    datatypes: (property)-[:datatype]->(datatype) \
    ' \
--where 'datatype in ["wikibase-item"]' \
--return 'distinct n as node1' \
/ cat -i - -i "$TEMP"/root.nodes.ontology.tsv.gz --mode=NONE \
/ compact --mode=NONE --columns node1 \
-o "$TEMP"/root.nodes.ontology.star.tsv.gz \

!$kypher -i "$TEMP"/root.nodes.ontology.star.tsv.gz --as rootnode1 --limit 2

[2021-10-10 02:52:36 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_24_c1."node2" "_aLias.node1"
     FROM graph_24 AS graph_24_c1
     INNER JOIN graph_4 AS graph_4_c2
     ON graph_4_c2."node1" = graph_24_c1."label"
        AND graph_24_c1."label" = graph_4_c2."node1"
        AND graph_4_c2."label" = ?
        AND (graph_4_c2."node2" IN (?))
  PARAS: ['datatype', 'wikibase-item']
---------------------------------------------
[2021-10-10 02:52:36 sqlstore]: CREATE INDEX on table graph_24 column label ...
[2021-10-10 02:52:37 sqlstore]: ANALYZE INDEX on table graph_24 column label ...
[2021-10-10 02:52:41 sqlstore]: DROP graph data table graph_27 from rootnode1
[2021-10-10 02:52:41 sqlstore]: IMPORT graph directly into table graph_27 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/temp.build-tutorial/root.nodes.ontology.star.tsv.gz ...
[2021-10-10 02:52:42 query]: SQL Translation:
----------------------------------------

Include in node1 all the properties in the graph

In [49]:
%%time
!$kypher -i rootbase \
--match ' \
    rootbase: ()-[l {label: property}]->(n)' \
--return 'distinct property as node1' \
/ cat -i - -i "$TEMP"/root.nodes.ontology.star.tsv.gz --mode=NONE \
/ compact --mode=NONE --columns node1 \
-o "$TEMP"/root.nodes.ontology.star.property.tsv.gz \

!$kypher -i "$TEMP"/root.nodes.ontology.star.property.tsv.gz --as rootnode1 --limit 2

[2021-10-10 02:52:43 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_24_c1."label" "_aLias.node1"
     FROM graph_24 AS graph_24_c1
     WHERE graph_24_c1."label" = graph_24_c1."label"
  PARAS: []
---------------------------------------------
[2021-10-10 02:52:46 sqlstore]: DROP graph data table graph_27 from rootnode1
[2021-10-10 02:52:46 sqlstore]: IMPORT graph directly into table graph_27 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/temp.build-tutorial/root.nodes.ontology.star.property.tsv.gz ...
[2021-10-10 02:52:47 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_27 AS graph_27_c1
     LIMIT ?
  PARAS: [2]
---------------------------------------------
node1
P10
P1000
CPU times: user 56.2 ms, sys: 26.8 ms, total: 83 ms
Wall time: 4.77 s


## Add property datatypes

In [50]:
%%time
!$kypher -i datatypes -i rootbase \
--match ' \
    rootbase: ()-[r {label: property}]->(), \
    datatypes: (property)-[l:datatype]->(datatype) \
    ' \
--return 'distinct property as node1, l.label as label, datatype as node2, l as id' \
/ cat -i - -i $OUT/root.graph.item.quantity.time.monolingual.string.property.qualifiers.P31.P279.ontology.tsv.gz \
/ compact \
-o $OUT/root.graph.item.quantity.time.monolingual.string.property.qualifiers.P31.P279.ontology.datatype.tsv.gz \

!$kypher -i root.graph.item.quantity.time.monolingual.string.property.qualifiers.P31.P279.ontology.datatype.tsv.gz --as rootbase --limit 2

[2021-10-10 02:52:47 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_4_c2."node1" "_aLias.node1", graph_4_c2."label" "_aLias.label", graph_4_c2."node2" "_aLias.node2", graph_4_c2."id" "_aLias.id"
     FROM graph_24 AS graph_24_c1
     INNER JOIN graph_4 AS graph_4_c2
     ON graph_4_c2."node1" = graph_24_c1."label"
        AND graph_24_c1."label" = graph_4_c2."node1"
        AND graph_4_c2."label" = ?
  PARAS: ['datatype']
---------------------------------------------
[2021-10-10 02:53:17 sqlstore]: DROP graph data table graph_24 from rootbase
[2021-10-10 02:53:18 sqlstore]: IMPORT graph directly into table graph_24 from /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/root.graph.item.quantity.time.monolingual.string.property.qualifiers.P31.P279.ontology.datatype.tsv.gz ...
[2021-10-10 02:53:26 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_24 AS graph_24_c1
     LIMIT ?
  PARA

## Build labels, aliases and descriptions

Extract the label edges

In [51]:
%%time
!$kypher -i label -i rootnode1 \
--match ' \
    rootnode1: (n1)-[]->(), \
    label: (n1)-[l]->(n2) \
    ' \
--return 'distinct n1 as node1, l.label as label, n2 as node2, l as id' \
/ sort \
-o $OUT/root.graph.label.tsv.gz

[2021-10-10 02:53:27 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_27_c1."node1" "_aLias.node1", graph_15_c2."label" "_aLias.label", graph_15_c2."node2" "_aLias.node2", graph_15_c2."id" "_aLias.id"
     FROM graph_15 AS graph_15_c2
     INNER JOIN graph_27 AS graph_27_c1
     ON graph_27_c1."node1" = graph_15_c2."node1"
  PARAS: []
---------------------------------------------
[2021-10-10 02:53:27 sqlstore]: CREATE INDEX on table graph_27 column node1 ...
[2021-10-10 02:53:27 sqlstore]: ANALYZE INDEX on table graph_27 column node1 ...
CPU times: user 230 ms, sys: 79.7 ms, total: 310 ms
Wall time: 20.7 s


Extract the alias edges

In [52]:
%%time
!$kypher -i alias -i rootnode1 \
--match ' \
    rootnode1: (n1)-[]->(), \
    alias: (n1)-[l]->(n2) \
    ' \
--return 'distinct n1 as node1, l.label as label, n2 as node2, l as id' \
/ sort \
-o $OUT/root.graph.alias.tsv.gz

[2021-10-10 02:53:47 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_27_c1."node1" "_aLias.node1", graph_16_c2."label" "_aLias.label", graph_16_c2."node2" "_aLias.node2", graph_16_c2."id" "_aLias.id"
     FROM graph_16 AS graph_16_c2
     INNER JOIN graph_27 AS graph_27_c1
     ON graph_27_c1."node1" = graph_16_c2."node1"
  PARAS: []
---------------------------------------------
CPU times: user 61.3 ms, sys: 22.3 ms, total: 83.6 ms
Wall time: 5.4 s


Extract the description edges

In [53]:
%%time
!$kypher -i description -i rootnode1 \
--match ' \
    rootnode1: (n1)-[]->(), \
    description: (n1)-[l]->(n2) \
    ' \
--return 'distinct n1 as node1, l.label as label, n2 as node2, l as id' \
/ sort \
-o $OUT/root.graph.description.tsv.gz

[2021-10-10 02:53:53 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_27_c1."node1" "_aLias.node1", graph_17_c2."label" "_aLias.label", graph_17_c2."node2" "_aLias.node2", graph_17_c2."id" "_aLias.id"
     FROM graph_17 AS graph_17_c2
     INNER JOIN graph_27 AS graph_27_c1
     ON graph_27_c1."node1" = graph_17_c2."node1"
  PARAS: []
---------------------------------------------
CPU times: user 243 ms, sys: 77.5 ms, total: 320 ms
Wall time: 21.1 s


## Compute useful derived files

### Inverses of `P279`

> To do: need to define t`P279_` property, it's datatype, label, etc.

In [54]:
!$kypher -i rootbase \
--match '(n1)-[:P279]->(class)' \
--return 'distinct class as node1, "P279_" as label, n1 as node2' \
/ add-id --id-style wikidata \
/ sort \
-o "$OUT"/root.derived.P279inv.tsv.gz

[2021-10-10 02:54:14 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_24_c1."node2" "_aLias.node1", ? "_aLias.label", graph_24_c1."node1" "_aLias.node2"
     FROM graph_24 AS graph_24_c1
     WHERE graph_24_c1."label" = ?
  PARAS: ['P279_', 'P279']
---------------------------------------------
[2021-10-10 02:54:14 sqlstore]: CREATE INDEX on table graph_24 column label ...
[2021-10-10 02:54:15 sqlstore]: ANALYZE INDEX on table graph_24 column label ...


## Final files
- base, includes all edges except labeles, aliases and descriptions
- labels
- aliases
- descriptions

In [55]:
%%time
!$kgtk cat \
-i "$OUT"/root.graph.item.quantity.time.monolingual.string.property.qualifiers.P31.P279.ontology.datatype.tsv.gz \
-i "$OUT"/root.graph.alias.tsv.gz \
-i "$OUT"/root.graph.label.tsv.gz \
-i "$OUT"/root.graph.description.tsv.gz \
-o "$OUT"/all.tsv.gz

CPU times: user 134 ms, sys: 47.2 ms, total: 181 ms
Wall time: 11.9 s
